# Sistemas Inteligentes 2021/2022

## Mini-projeto 2: Quadrados Latinos


## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias
Número:  51127       Nome:   Luís Ferreirinha


_o 3º colega não participou em nada_

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação de variáveis, domínios, vizinhos e restrições

O problema dos quadrados latinos pode ser representado como uma matriz $n \times n$ em que cada linha e coluna contêm exatmente um número de $1$ a $n$.  

Então para caracterizar este CSP em python obtamos pelas seguintes escolhas:  

- Variáveis:  
  
    Cada variável no nosso problema vai representar o valor atribuiado a um elemento da matriz.  
    Então em python representamos o conjunto das variáveis como uma lista das coordenadas de cada
    elemento na matriz:    
    -  var $=[(i_1,j_1),...,(i_n,j_n)]$  
<br>
- Domínios:  
  
    Neste problema cada variável pode tomar um valor de $1,...,n$, exceto de houver condições iniciais.
    Portanto em python decidimos utilizar um dicionário, onde as chaves são as variáveis e estas correspondem a uma lista de inteiros contento os valores possíveis para aquela variável:
    
    - dominio $=\{(i_1,j_1) = [1,..,n], ...\}$  
<br> 
- Vizinhos:  

    Uma variável vai ser vizinha de outra de estiver na mesma linha ou coluna que essa variável.
    Em python definimos os vizinhos como um dicionário onde cada variável é uma chave e corresponde a uma lista de variáveis que são suas vizinhas:  
    
    - vizinhos $=\{(i_k,i_m) : [(i_1,j_m),...,(i_n,j_m),(i_k,j_1),...,(i_k,j_n)], \, ...  \, \}$  
<br>
- Restrições:  

    A restrição principal deste problema é a impossibilidade de haver repetições de um número
    ao longo de uma coluna ou linha, ou seja tem de ser diferente de todos os seus vizinhos.
    Em python esta restrição é representada pela comparação dos valores de duas variáveis vizinhas:  

    -  ```python
       valor_variavel_1 != valor_variavel_2
       ```
    <br>
    Como também temos a variação do Futoshiki, temos de definir outras restrições para esse problema.  

    A ideia principal é que duas variáveis adjacentes podem agora ter a restrição adicional de o valor de uma ser maior ou menor que o valor da outra. Então para representar o mapeamento destas novas restrições usamos um dicionário onde as chaves são um tuplos contendos as duas variáveis em questão e vão corresponde a uma função que verifica se os valores destas são menores ou maiores conforme a restrição.

    - desigualdades $=\{((i_k,j_m),(i_k,j_{m+1})) : maior\, \, \text{ou} \, \, menor \}$  

    Cada uma destas funções vair comparar os valores das variavéis conforme o seu nome.  

    - ```python 
      def maior(a,b): return a > b
      ```

    - ```python 
      def menor(a,b): return a < b
      ```


 

## Formulação do problema

In [80]:
from csp import *

def quadrado_latino(n=3, initial={}) -> CSP:
    """Gera um CSP para o problema dos quadrados latinos

    Args:
        n (int, optional): Dimensao da matriz do problema. Defaults to 3.
        initial (dict, optional): Valores iniciais para cada um dos elementos da matriz. Defaults to {}.

    Returns:
        CSP: Objeto CSP para o problema dos quadrados latinos
    """    
    
    # definicao das variaveis, as coordenadas x,y vão de 0 a n-1
    variaveis = [(i,j) for i in range(0,n) for j in range(0,n)]

    # Valores possiveis para as variaveis dada a dimensao n do problema
    conjunto = [i for i in range(1,n+1)]
    
    # Iniciamos os dominios com os valores iniciais dados para as variaveis
    dominios = initial

    # Diferenca dos dois conjuntos vai resultar nas variaveis nao preenchidas
    for item in set(variaveis - initial.keys()):
        dominios[item] = conjunto
    
    # Condicao que determina de duas variaveis sao vizinhas, verifica se tem alguma das coordendas x ou y iguais
    condicao_vizinhanca = lambda x,y : (x[0] == y[0] and x[1] != y[1]) or (x[0] != y[0] and x[1] == y[1])

    # Para cada variavel vamos atribuir uma lista resultante de filtrar a lista de todas as variaveis
    # com a condicao de que as variaveis sejam vizinhas da variavel chave
    vizinhos = {var : list(filter(lambda x: condicao_vizinhanca(x,var), variaveis)) for var in variaveis}

    def restricoes(X, a, Y, b):
        # Verifica-se se duas variaveis sao vizinhas antes de verficar que tem valores diferentes entre si
        if Y in vizinhos[X]:
            return a != b
        return True
        
    return CSP(variaveis, dominios, vizinhos, restricoes)

In [81]:
# Funcoes restricao
def maior(x,y):
    return x > y

def menor(x,y):
    return x < y

def futoshiki(n=3, initial={}, desigualdades={}):
    """Gera um CSP para o problema do futoshiki

    Args:
        n (int, optional): Dimensao da matriz do problema. Defaults to 3.
        initial (dict, optional): Valores iniciais para cada um dos elementos da matriz. Defaults to {}.
        desigualdades (dict, optional): Restrições adicionais do problema futoshiki. Defaults to {}.

    Returns:
        CSP: Objeto CSP para o problema dos quadrados latinos
    """    
    
    # definicao das variaveis, as coordenadas x,y vão de 0 a n-1
    variaveis = [(i,j) for i in range(0,n) for j in range(0,n)]

    # Valores possiveis para as variaveis dada a dimensao n do problema
    conjunto = [i for i in range(1,n+1)]
    
    # Iniciamos os dominios com os valores iniciais dados para as variaveis
    dominios = initial

    # Diferenca dos dois conjuntos vai resultar nas variaveis nao preenchidas
    for item in set(variaveis - initial.keys()):
        dominios[item] = conjunto
    
    # Condicao que determina de duas variaveis sao vizinhas, verifica se tem alguma das coordendas x ou y iguais
    condicao_vizinhanca = lambda x,y : (x[0] == y[0] and x[1] != y[1]) or (x[0] != y[0] and x[1] == y[1])

    # Para cada variavel vamos atribuir uma lista resultante de filtrar a lista de todas as variaveis
    # com a condicao de que as variaveis sejam vizinhas da variavel chave
    vizinhos = {var : list(filter(lambda x: condicao_vizinhanca(x,var), variaveis)) for var in variaveis}

    #for var1,var2 in desigualdades.keys():
    #    if var2 not in vizinhos[var1]:
    #        vizinhos[var1].append(var2)
    #    if var1 not in vizinhos[var2]:
    #        vizinhos[var2].append(var1)

    def restricoes(X, a, Y, b):
        # Vamos verificar se a var X tem alguma condição de desigualdade com outra var Y
        if (X,Y) in desigualdades.keys():
            if not desigualdades[(X,Y)](a,b):
                return False
        # Como na definicao das condicoes temos soh num sentido temos de verificar no sentido
        # contrario aqui tambem
        if (Y,X) in desigualdades.keys():
            if not desigualdades[(Y,X)](b,a):
                return False
        # Verifica se sao vizinhos
        if Y in vizinhos[X]:
            return  a != b
        return True

    return CSP(variaveis, dominios, vizinhos, restricoes)

## Criação do problema do quadrado latino simples

Mostrem que o código está a funcionar, construindo um problema de quadrado latino *4x4*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Mostre como podemos criar um puzzle com quadrados já preenchidos, e qual o impacto que isso tem nas variáveis, domínios iniciais, e vizinhos.

Para criar um problema quadrado latino 4x4 sem valores iniciais para as variáveis basta executar

In [82]:
q4 = quadrado_latino(4)

Tendo o problema definido, podemos agora ver as variáveis, domínios e os vizinhos deste problema

In [83]:
print(f"Variáveis = {q4.variables}\n")
print(f"Domínios = {q4.domains}\n")
print(f"Vizinhos = {q4.neighbors}\n")

Variáveis = [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]

Domínios = {(0, 1): [1, 2, 3, 4], (1, 2): [1, 2, 3, 4], (2, 1): [1, 2, 3, 4], (3, 1): [1, 2, 3, 4], (0, 2): [1, 2, 3, 4], (2, 2): [1, 2, 3, 4], (1, 0): [1, 2, 3, 4], (3, 2): [1, 2, 3, 4], (1, 3): [1, 2, 3, 4], (0, 0): [1, 2, 3, 4], (1, 1): [1, 2, 3, 4], (0, 3): [1, 2, 3, 4], (2, 0): [1, 2, 3, 4], (3, 0): [1, 2, 3, 4], (2, 3): [1, 2, 3, 4], (3, 3): [1, 2, 3, 4]}

Vizinhos = {(0, 0): [(0, 1), (0, 2), (0, 3), (1, 0), (2, 0), (3, 0)], (0, 1): [(0, 0), (0, 2), (0, 3), (1, 1), (2, 1), (3, 1)], (0, 2): [(0, 0), (0, 1), (0, 3), (1, 2), (2, 2), (3, 2)], (0, 3): [(0, 0), (0, 1), (0, 2), (1, 3), (2, 3), (3, 3)], (1, 0): [(0, 0), (1, 1), (1, 2), (1, 3), (2, 0), (3, 0)], (1, 1): [(0, 1), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1)], (1, 2): [(0, 2), (1, 0), (1, 1), (1, 3), (2, 2), (3, 2)], (1, 3): [(0, 3), (1, 0), (1, 1), (1, 2), (2, 3), (3, 3)], (2, 0): [(0, 0), 

Para criar-mos um problema com um quadrado semi-preenchido temos primeiro de definir quais os elementos da matriz que vão estar preenchidos e qual o seu valor. Para fazer-mos isso podemos definir um dicionário com essa informação da seguinte forma:

In [84]:
valores_preenchidos = {(0,0) : [1], (1,1) : [3], (2,2) : [2]}

Depois basta passar esse dicionário como segundo argumento à função quadrado_latino

In [85]:
q4_semi = quadrado_latino(n=4, initial=valores_preenchidos)

O facto de o quadrado ser semi preenchido apenas vai afetar os domínios iniciais, fazendo com que o dominio inicial das variáveis já com valores atribuidos seja reduzido esses valores.

In [86]:
print(f"Variáveis = {q4_semi.variables}\n")
print(f"Domínios = {q4_semi.domains}\n")
print(f"Vizinhos = {q4_semi.neighbors}\n")

Variáveis = [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]

Domínios = {(0, 0): [1], (1, 1): [3], (2, 2): [2], (0, 1): [1, 2, 3, 4], (1, 2): [1, 2, 3, 4], (2, 1): [1, 2, 3, 4], (3, 1): [1, 2, 3, 4], (0, 3): [1, 2, 3, 4], (2, 0): [1, 2, 3, 4], (3, 0): [1, 2, 3, 4], (2, 3): [1, 2, 3, 4], (0, 2): [1, 2, 3, 4], (3, 3): [1, 2, 3, 4], (1, 0): [1, 2, 3, 4], (3, 2): [1, 2, 3, 4], (1, 3): [1, 2, 3, 4]}

Vizinhos = {(0, 0): [(0, 1), (0, 2), (0, 3), (1, 0), (2, 0), (3, 0)], (0, 1): [(0, 0), (0, 2), (0, 3), (1, 1), (2, 1), (3, 1)], (0, 2): [(0, 0), (0, 1), (0, 3), (1, 2), (2, 2), (3, 2)], (0, 3): [(0, 0), (0, 1), (0, 2), (1, 3), (2, 3), (3, 3)], (1, 0): [(0, 0), (1, 1), (1, 2), (1, 3), (2, 0), (3, 0)], (1, 1): [(0, 1), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1)], (1, 2): [(0, 2), (1, 0), (1, 1), (1, 3), (2, 2), (3, 2)], (1, 3): [(0, 3), (1, 0), (1, 1), (1, 2), (2, 3), (3, 3)], (2, 0): [(0, 0), (1, 0), (2, 1), (2, 2), (2,

Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística.

Como a procura para um problema de dimensão baixa demora muito pouco tempo (ordem dos $1\times10^{-4}\, s$) Vamos definir uma função que executa uma pesquisa $n$ vezes (neste caso 50) e calcula a média dos tempos de execução da pesquisa. Desta forma podemos obter tempos de execução mais consistentes que não vão depender tanto de variações na velocidade do processador durante a execução das diferentes procuras.

In [87]:
import timeit

def avg_run_time(func, *args, **kwargs):
    """Corre uma função n vezes medindo o tempo de execução de cada vez
    e imprime a média dos tempos

    Args:
        func: Função a correr
    """
    n = 20
    run_times = []
    for i in range(n):
        start = timeit.default_timer()
        func(*args, **kwargs)
        stop = timeit.default_timer()
        run_times.append(stop-start)
    return sum(run_times) / n

# Guardar os dados e solucoes para analisar mais tarde
tempos = {}
solucoes = {}

- Dimensão do problema:

In [88]:
n = 4

- Backtracking sem inferência

In [89]:
# Definir problema
QuadradoLatinoSemInferência = quadrado_latino(n)
# Guardar solucao
sol_1 = backtracking_search(QuadradoLatinoSemInferência)
# Calcular tempo medio executando 20 vezes
tempo_medio1 = avg_run_time(backtracking_search, QuadradoLatinoSemInferência)
# Guardar resultados
tempos['QuadradoLatinoSemInferência'] = tempo_medio1
solucoes['QuadradoLatinoSemInferência'] = sol_1
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio1}")
print('Assignment (QuadradoLatinoSemInferência) = ',sol_1)

Tempo médio de execução: 0.0005906399979721755
Assignment (QuadradoLatinoSemInferência) =  {(0, 0): 1, (0, 1): 2, (0, 2): 3, (0, 3): 4, (1, 0): 2, (1, 1): 1, (1, 2): 4, (1, 3): 3, (2, 0): 3, (2, 1): 4, (2, 2): 1, (2, 3): 2, (3, 0): 4, (3, 1): 3, (3, 2): 2, (3, 3): 1}


- Backtracking com inferência Foward-Checking

In [90]:
# Definir problema
QuadradoLatinoComInferência_FRWDchecking = quadrado_latino(n)
# Resolver e guardar solucao
sol_2 = backtracking_search(QuadradoLatinoComInferência_FRWDchecking, inference=forward_checking)
# Calcular o tempo medio de execucao
tempo_medio2 = avg_run_time(backtracking_search, QuadradoLatinoComInferência_FRWDchecking, inference=forward_checking)
# Guardar resultados
tempos['QuadradoLatinoComInferência_FRWDchecking'] = tempo_medio2
solucoes['QuadradoLatinoComInferência_FRWDchecking'] = sol_2
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio2}")
print('Assignment (QuadradoLatinoComInferência_FRWDchecking) = ',sol_2)

Tempo médio de execução: 0.0006393249903339892
Assignment (QuadradoLatinoComInferência_FRWDchecking) =  {(0, 0): 1, (0, 1): 2, (0, 2): 3, (0, 3): 4, (1, 0): 2, (1, 1): 1, (1, 2): 4, (1, 3): 3, (2, 0): 3, (2, 1): 4, (2, 2): 1, (2, 3): 2, (3, 0): 4, (3, 1): 3, (3, 2): 2, (3, 3): 1}


- Backtracking com inferência MAC

In [91]:
# Definir problema
QuadradoLatinoComInferência_MAC = quadrado_latino(n)
# Resolver e guardar solucao
sol_4 = backtracking_search(QuadradoLatinoComInferência_MAC, inference=mac)
# Calcular tempo medio de execucao
tempo_medio4 = avg_run_time(backtracking_search, QuadradoLatinoComInferência_MAC, inference=mac)
# Guarda resultados
tempos['QuadradoLatinoComInferência_MAC'] = tempo_medio4
solucoes['QuadradoLatinoComInferência_MAC'] = sol_4
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio4}")
print('Assignment (QuadradoLatinoComInferência_MAC) = ', sol_4)

Tempo médio de execução: 0.0006703250110149384
Assignment (QuadradoLatinoComInferência_MAC) =  {(0, 0): 1, (0, 1): 2, (0, 2): 3, (0, 3): 4, (1, 0): 2, (1, 1): 1, (1, 2): 4, (1, 3): 3, (2, 0): 3, (2, 1): 4, (2, 2): 1, (2, 3): 2, (3, 0): 4, (3, 1): 3, (3, 2): 2, (3, 3): 1}


- Backtracking com pré-processamento AC3

In [92]:
# Definir problema
QuadradoLatinoComAC3 = quadrado_latino(n)
# Aplicar pre-processamento
p_AC3 = AC3(QuadradoLatinoComAC3)
# Resolver e guardar solucao
sol_3 = backtracking_search(p_AC3)
# Calcular o tempo medio de execucao
tempo_medio3 = avg_run_time(backtracking_search, p_AC3)
# Guarda resultados
tempos['QuadradoLatinoComAC3'] = tempo_medio3
solucoes['QuadradoLatinoComAC3'] = sol_3
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio3}")
print('Assignment (QuadradoLatinoComAC3) = ', sol_3)

Tempo médio de execução: 0.0004470299929380417
Assignment (QuadradoLatinoComAC3) =  {(0, 0): 1, (0, 1): 2, (0, 2): 3, (0, 3): 4, (1, 0): 2, (1, 1): 1, (1, 2): 4, (1, 3): 3, (2, 0): 3, (2, 1): 4, (2, 2): 1, (2, 3): 2, (3, 0): 4, (3, 1): 3, (3, 2): 2, (3, 3): 1}


- Backtracking com Heurística MRV

In [93]:
# Definir problema
QuadradoLatinoComHeuristica = quadrado_latino(n)
# Resolver e guardar
sol_6 = backtracking_search(QuadradoLatinoComHeuristica, select_unassigned_variable = mrv)
# Tempo medio
tempo_medio6 = avg_run_time(backtracking_search, QuadradoLatinoComHeuristica, select_unassigned_variable = mrv)
# Guarda resultados
tempos['QuadradoLatinoComHeuristica'] = tempo_medio6
solucoes['QuadradoLatinoComHeuristica'] = sol_6
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio6}")
print('Assignment (QuadradoLatinoComHeuristica) = ', sol_6)

Tempo médio de execução: 0.0011016850010491907
Assignment (QuadradoLatinoComHeuristica) =  {(0, 1): 1, (3, 1): 2, (2, 1): 3, (0, 2): 2, (1, 0): 1, (1, 3): 2, (3, 3): 1, (1, 2): 3, (2, 0): 2, (1, 1): 4, (3, 2): 4, (0, 0): 4, (0, 3): 3, (3, 0): 3, (2, 3): 4, (2, 2): 1}


- Backtracking com Heurística MRV, Inferência foward_checking e pré-processamento AC3

In [94]:
# Definir problema
QuadradoLatinoHeuristicaInferenciaAC3 = quadrado_latino(n)
# Pre processamento
p_AC3 = AC3(QuadradoLatinoHeuristicaInferenciaAC3)
# Resolver e guardar
sol_7 = backtracking_search(p_AC3, select_unassigned_variable=mrv, inference=mac)
# Tempo medio
tempo_medio7 = avg_run_time(backtracking_search, p_AC3, select_unassigned_variable=mrv, inference=mac)
# Guarda resultados
tempos['QuadradoLatinoHeuristicaInferenciaAC3'] = tempo_medio7
solucoes['QuadradoLatinoHeuristicaInferenciaAC3'] = sol_7
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio7}")
print('Assignment (QuadradoLatinoHeuristicaInferenciaAC3) = ', sol_7)

Tempo médio de execução: 0.0015277399972546846
Assignment (QuadradoLatinoHeuristicaInferenciaAC3) =  {(0, 2): 1, (3, 2): 2, (2, 2): 3, (1, 2): 4, (0, 3): 2, (1, 3): 1, (3, 3): 3, (2, 3): 4, (0, 1): 3, (3, 0): 1, (2, 1): 1, (3, 1): 4, (2, 0): 2, (1, 1): 2, (0, 0): 4, (1, 0): 3}


In [95]:
import pandas as pd
temposSorted = dict(sorted(tempos.items(), key=lambda x:x[1]))
pd.DataFrame(temposSorted.values(),index = temposSorted.keys(), columns= ["Tempos"])


,Tempos
QuadradoLatinoComAC3,0.000447
QuadradoLatinoSemInferência,0.000591
QuadradoLatinoComInferência_FRWDchecking,0.000639
QuadradoLatinoComInferência_MAC,0.000670
QuadradoLatinoComHeuristica,0.001102
QuadradoLatinoHeuristicaInferenciaAC3,0.001528


## Conclusões sobre os resultados temporais
_(todos os valores são referentes a médias)_
Com a tabela acima é possível concluir que:
- o algoritmo que demora menos tempo a encontrar uma solução é a **procura com Retrocesso usando a inferência Forward Checking**, demora apenas 0.000219 segundos;
- usando procura com retrocesso com inferência, verifica-se que **Forward Checking** é mais eficiente que **MAC**;
- a solução do problema utilizando a procura com retrocesso sem inferências nem heurísticas demora menos tempo (0.000337 segundos) que a procura com retrocesso com inferência e heurisrica após pré-processamento com recurso ao algoritmo **AC3** (0.000623 segundos), quase o dobro do tempo;


In [96]:
sols = pd.DataFrame(solucoes).T
sols

0           1           2           \
                                          0  1  2  3  0  1  2  3  0  1  2  3   
QuadradoLatinoSemInferência               1  2  3  4  2  1  4  3  3  4  1  2   
QuadradoLatinoComInferência_FRWDchecking  1  2  3  4  2  1  4  3  3  4  1  2   
QuadradoLatinoComInferência_MAC           1  2  3  4  2  1  4  3  3  4  1  2   
QuadradoLatinoComAC3                      1  2  3  4  2  1  4  3  3  4  1  2   
QuadradoLatinoComHeuristica               4  1  2  3  1  4  3  2  2  3  1  4   
QuadradoLatinoHeuristicaInferenciaAC3     4  3  1  2  3  2  4  1  2  1  3  4   

                                          3           
                                          0  1  2  3  
QuadradoLatinoSemInferência               4  3  2  1  
QuadradoLatinoComInferência_FRWDchecking  4  3  2  1  
QuadradoLatinoComInferência_MAC           4  3  2  1  
QuadradoLatinoComAC3                      4  3  2  1  
QuadradoLatinoComHeuristica               3  2  4  1  
QuadradoLatinoHeuristicaInferenciaAC3     1  4  2  3

In [97]:
sols.drop_duplicates()

0           1           2           3  \
                                       0  1  2  3  0  1  2  3  0  1  2  3  0   
QuadradoLatinoSemInferência            1  2  3  4  2  1  4  3  3  4  1  2  4   
QuadradoLatinoComHeuristica            4  1  2  3  1  4  3  2  2  3  1  4  3   
QuadradoLatinoHeuristicaInferenciaAC3  4  3  1  2  3  2  4  1  2  1  3  4  1   

                                                
                                       1  2  3  
QuadradoLatinoSemInferência            3  2  1  
QuadradoLatinoComHeuristica            2  4  1  
QuadradoLatinoHeuristicaInferenciaAC3  4  2  3

## Conclusões sobre os resultados de soluções

Podemos concluir que usando a heurística MRV a solução do problema é diferente quando não a usamos. Mais ainda, verifica-se que mesmo usando a heurística MRV, a solução do problema é diferente quando se fez um pré-processamento com recurso ao algoritmo AC3.



## Visualização do problema

In [98]:
def visualizacao_quadrado_latino(n, problema_csp, solucao):
    """Gera uma visualização do estado inicial e da solução de um problema de quadrados
    latinos de dimensão n
    
    Args:
        n (int): Dimensão do problema
        problema_csp (CSP): Object CSP do problema em questão
        solucao (dict): Solução do problema
    """    
    
    # Construir matriz inicial com os valores do problema
    estado_inicial = [[0 for y in range(n)] for x in range(n)]
    for var, dom in problema_csp.domains.items():
        if len(dom) == 1:
            estado_inicial[var[1]][var[0]] = dom[0]
        
    # Construir matriz final com os valores do problema
    estado_final = [[0 for y in range(n)] for x in range(n)]
    for var, valor in solucao.items():
        estado_final[var[1]][var[0]] = valor
    
    # Construir strings que representam as matrizes
    board_inicial = ""
    board_final = ""
    for y in range(n):
        for x in range(n):
            board_inicial += str(estado_inicial[y][x]) + " "
            board_final += str(estado_final[y][x]) + " "
        board_inicial += "\n"
        board_final += "\n"
    
    print("Estado Inicial:")
    print(board_inicial)
    print("Solução:")
    print(board_final)

Para visualizar basta invocar a função visualizacao_quadrado_latino dando a dimensao, o CSP e a solução como argumentos

In [99]:
visualizacao_quadrado_latino(n, quadrado_latino(n), backtracking_search(quadrado_latino(n)))

Estado Inicial:
0 0 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 

Solução:
1 2 3 4 
2 1 4 3 
3 4 1 2 
4 3 2 1 



## Criação do problema Futoshiki *5x5*

Mostrem que o código está a funcionar, construindo um problema de Futoshiki *5x5*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Utilize o [link](https://www.futoshiki.org/) para gerar puzzles e validar a implementação.

Para criar um problema Futoshiki vamos ter definir os elementos da matriz que estão preenchidos e estabelecer as relações de desigualdade presentes no problema.

In [65]:
preenchidos5 = {(0,3) : [2], (4,3) : [3]}
desigualdades5 = {((3,0),(4,0)) : maior, ((0,1),(1,1)) : menor, ((0,2),(1,2)) : menor,\
                 ((3,3),(3,2)) : menor, ((1,4),(2,4)) : maior, ((2,4),(2,3)) : maior, \
                 ((4,4),(4,3)) : maior}

Podemos então criar um problema Futoshiki 5x5 da seguinte forma

In [100]:
futoshiki5 = futoshiki(5, preenchidos5, desigualdades5)

In [101]:
print("Variáveis = ", futoshiki5.variables, "\n")
print("Domínios = ", futoshiki5.domains,"\n")
print("Vizinhos = ", futoshiki5.neighbors, "\n")

Variáveis =  [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)] 

Domínios =  {(0, 3): [2], (4, 3): [3], (0, 1): [1, 2, 3, 4, 5], (2, 4): [1, 2, 3, 4, 5], (4, 0): [1, 2, 3, 4, 5], (1, 2): [1, 2, 3, 4, 5], (3, 4): [1, 2, 3, 4, 5], (0, 4): [1, 2, 3, 4, 5], (2, 1): [1, 2, 3, 4, 5], (3, 1): [1, 2, 3, 4, 5], (0, 2): [1, 2, 3, 4, 5], (2, 2): [1, 2, 3, 4, 5], (1, 0): [1, 2, 3, 4, 5], (3, 2): [1, 2, 3, 4, 5], (1, 3): [1, 2, 3, 4, 5], (4, 1): [1, 2, 3, 4, 5], (4, 4): [1, 2, 3, 4, 5], (0, 0): [1, 2, 3, 4, 5], (1, 1): [1, 2, 3, 4, 5], (2, 0): [1, 2, 3, 4, 5], (4, 2): [1, 2, 3, 4, 5], (3, 0): [1, 2, 3, 4, 5], (1, 4): [1, 2, 3, 4, 5], (2, 3): [1, 2, 3, 4, 5], (3, 3): [1, 2, 3, 4, 5]} 

Vizinhos =  {(0, 0): [(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (2, 0), (3, 0), (4, 0)], (0, 1): [(0, 0), (0, 2), (0, 3), (0, 4), (1, 1), (2, 1), (3, 1), (4, 1)], (0, 2):

Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística. Até que dimensão consegue resolver o problema em menos de 1 minuto?

In [113]:
solucoesFutoshiki = {}
temposFutoshiki = {}

- Backtracking sem inferência

In [114]:
# Definir problema
Futoshiki5SemInferência = futoshiki(5, preenchidos5, desigualdades5)
# Guardar solucao
sol_15 = backtracking_search(Futoshiki5SemInferência)
# Calcular tempo medio executando 20 vezes
tempo_medio15 = avg_run_time(backtracking_search,Futoshiki5SemInferência)
# Guardar resultados
temposFutoshiki['Futoshiki5SemInferência'] = tempo_medio15
solucoesFutoshiki['Futoshiki5SemInferência'] = sol_15
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio15}")
print('Assignment (Futoshiki5SemInferência) = ',sol_15)

Tempo médio de execução: 0.0032846299931406976
Assignment (Futoshiki5SemInferência) =  {(0, 0): 5, (0, 1): 3, (0, 2): 1, (0, 3): 2, (0, 4): 4, (1, 0): 1, (1, 1): 4, (1, 2): 2, (1, 3): 5, (1, 4): 3, (2, 0): 4, (2, 1): 5, (2, 2): 3, (2, 3): 1, (2, 4): 2, (3, 0): 3, (3, 1): 2, (3, 2): 5, (3, 3): 4, (3, 4): 1, (4, 0): 2, (4, 1): 1, (4, 2): 4, (4, 3): 3, (4, 4): 5}


- Backtracking com inferência Forward Checking

In [115]:
# Definir problema
Futoshiki5ComInferência_FRWDchecking = futoshiki(5, preenchidos5, desigualdades5)
# Resolver e guardar solucao
sol_25 = backtracking_search(Futoshiki5ComInferência_FRWDchecking, inference=forward_checking)
# Calcular o tempo medio de execucao
tempo_medio25 = avg_run_time(backtracking_search, Futoshiki5ComInferência_FRWDchecking, inference=forward_checking)
# Guardar resultados
temposFutoshiki['Futoshiki5ComInferência_FRWDchecking'] = tempo_medio25
solucoesFutoshiki['Futoshiki5ComInferência_FRWDchecking'] = sol_25
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio25}")
print('Assignment (Futoshiki5ComInferência_FRWDchecking) = ',sol_25)

Tempo médio de execução: 0.0019206600089091807
Assignment (Futoshiki5ComInferência_FRWDchecking) =  {(0, 0): 5, (0, 1): 3, (0, 2): 1, (0, 3): 2, (0, 4): 4, (1, 0): 1, (1, 1): 4, (1, 2): 2, (1, 3): 5, (1, 4): 3, (2, 0): 4, (2, 1): 5, (2, 2): 3, (2, 3): 1, (2, 4): 2, (3, 0): 3, (3, 1): 2, (3, 2): 5, (3, 3): 4, (3, 4): 1, (4, 0): 2, (4, 1): 1, (4, 2): 4, (4, 3): 3, (4, 4): 5}


- Backtracking com inferência MAC


In [116]:
# Definir problema
Futoshiki5ComInferência_MAC = futoshiki(5, preenchidos5, desigualdades5)
# Resolver e guardar solucao
sol_35 = backtracking_search(Futoshiki5ComInferência_MAC, inference=mac)
# Calcular o tempo medio de execucao
tempo_medio35 = avg_run_time(backtracking_search, Futoshiki5ComInferência_MAC, inference=mac)
# Guardar resultados
temposFutoshiki['Futoshiki5ComInferência_MAC'] = tempo_medio35
solucoesFutoshiki['Futoshiki5ComInferência_MAC'] = sol_35
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio35}")
print('Assignment (Futoshiki5ComInferência_MAC) = ',sol_35)

Tempo médio de execução: 0.004172584996558726
Assignment (Futoshiki5ComInferência_MAC) =  {(0, 0): 5, (0, 1): 3, (0, 2): 1, (0, 3): 2, (0, 4): 4, (1, 0): 1, (1, 1): 4, (1, 2): 2, (1, 3): 5, (1, 4): 3, (2, 0): 4, (2, 1): 5, (2, 2): 3, (2, 3): 1, (2, 4): 2, (3, 0): 3, (3, 1): 2, (3, 2): 5, (3, 3): 4, (3, 4): 1, (4, 0): 2, (4, 1): 1, (4, 2): 4, (4, 3): 3, (4, 4): 5}


- Backtracking com pré-processamento AC3

In [117]:
# Definir problema
Futoshiki5ComAC3 = futoshiki(5, preenchidos5, desigualdades5)
# Aplicar pre-processamento
p_AC3 = AC3(Futoshiki5ComAC3)
# Resolver e guardar solucao
sol_45 = backtracking_search(p_AC3)
# Calcular o tempo medio de execucao
tempo_medio45 = avg_run_time(backtracking_search, p_AC3)
# Guarda resultados
temposFutoshiki['Futoshiki5ComInferência_AC3'] = tempo_medio45
solucoesFutoshiki['Futoshiki5ComInferência_AC3'] = sol_45
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio45}")
print('Assignment (Futoshiki5ComInferência_AC3) = ',sol_45)


Tempo médio de execução: 0.0012719399994239212
Assignment (Futoshiki5ComInferência_AC3) =  {(0, 0): 5, (0, 1): 3, (0, 2): 1, (0, 3): 2, (0, 4): 4, (1, 0): 1, (1, 1): 4, (1, 2): 2, (1, 3): 5, (1, 4): 3, (2, 0): 4, (2, 1): 5, (2, 2): 3, (2, 3): 1, (2, 4): 2, (3, 0): 3, (3, 1): 2, (3, 2): 5, (3, 3): 4, (3, 4): 1, (4, 0): 2, (4, 1): 1, (4, 2): 4, (4, 3): 3, (4, 4): 5}


- Backtracking com Heurística MRV

In [118]:
# Definir problema
Futoshiki5ComHeuristica = futoshiki(5, preenchidos5, desigualdades5)
# Resolver e guardar solucao
sol_55 = backtracking_search(Futoshiki5ComHeuristica, select_unassigned_variable=mrv)
# Calcular o tempo medio de execucao
tempo_medio55 = avg_run_time(backtracking_search, Futoshiki5ComHeuristica, select_unassigned_variable=mrv)
# Guardar resultados
temposFutoshiki['Futoshiki5ComHeuristica'] = tempo_medio55
solucoesFutoshiki['Futoshiki5ComHeuristica'] = sol_55
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio55}")
print('Assignment (Futoshiki5ComHeuristica) = ',sol_55)

Tempo médio de execução: 0.0013626999920234085
Assignment (Futoshiki5ComHeuristica) =  {(4, 3): 3, (0, 3): 2, (2, 4): 2, (2, 2): 3, (4, 0): 2, (3, 3): 4, (1, 1): 4, (2, 1): 5, (3, 4): 1, (0, 0): 5, (4, 1): 1, (0, 1): 3, (3, 1): 2, (2, 3): 1, (4, 4): 5, (3, 0): 3, (1, 3): 5, (4, 2): 4, (0, 4): 4, (1, 2): 2, (1, 4): 3, (2, 0): 4, (1, 0): 1, (0, 2): 1, (3, 2): 5}


- Backtracking com Heurística MRV , inferência Forward Checking e pré-processamento AC3

In [119]:
# Definir problema
Futoshiki5ComHeuristica_FRWDchecking_AC3 = futoshiki(5, preenchidos5, desigualdades5)
# Aplicar pre-processamento
p_AC3 = AC3(Futoshiki5ComHeuristica_FRWDchecking_AC3)
# Resolver e guardar solucao
sol_65 = backtracking_search(p_AC3, select_unassigned_variable=mrv, inference=forward_checking)
# Calcular o tempo medio de execucao
tempo_medio65 = avg_run_time(backtracking_search, p_AC3, select_unassigned_variable=mrv, inference=forward_checking)
# Guardar resultados
temposFutoshiki['Futoshiki5ComHeuristica_FRWDchecking_AC3'] = tempo_medio65
solucoesFutoshiki['Futoshiki5ComHeuristica_FRWDchecking_AC3'] = sol_65
# Imprimir resultados
print(f"Tempo médio de execução: {tempo_medio65}")
print('Assignment (Futoshiki5ComHeuristica_FRWDchecking_AC3) = ',sol_65)


Tempo médio de execução: 0.001908900000853464
Assignment (Futoshiki5ComHeuristica_FRWDchecking_AC3) =  {(4, 3): 3, (2, 3): 1, (3, 3): 4, (3, 2): 5, (0, 3): 2, (1, 3): 5, (0, 1): 3, (0, 2): 1, (1, 1): 4, (1, 4): 3, (2, 4): 2, (3, 4): 1, (1, 2): 2, (4, 2): 4, (4, 4): 5, (3, 1): 2, (2, 2): 3, (4, 1): 1, (2, 1): 5, (4, 0): 2, (2, 0): 4, (0, 4): 4, (1, 0): 1, (0, 0): 5, (3, 0): 3}


In [121]:
pd.DataFrame(temposFutoshiki, index=['Tempo médio de execução']).T

,Tempo médio de execução
Futoshiki5SemInferência,0.003285
Futoshiki5ComInferência_FRWDchecking,0.001921
Futoshiki5ComInferência_MAC,0.004173
Futoshiki5ComInferência_AC3,0.001272
Futoshiki5ComHeuristica,0.001363
Futoshiki5ComHeuristica_FRWDchecking_AC3,0.001909


Futoshiki 7x7

In [112]:
preenchidos7ext = {(3,2) : [2], (4,0) : [4]}
desigualdades7ext = {((1,0),(2,0)):menor, ((2,0),(3,0)):menor, ((5,0),(6,0)): maior, ((2,1),(2,0)):menor, ((5,1),(5,0)):menor, ((0,1),(1,1)):maior, ((1,1),(2,1)):maior, ((2,1),(3,1)): menor, ((4,1),(5,1)):menor, ((2,2),(2,1)): menor, ((5,2),(5,1)): maior, ((1,3),(1,2)): menor, ((2,3),(2,2)): maior, ((0,3),(1,3)):maior, ((2,3),(3,3)):menor, ((4,3),(5,3)): menor, ((0,4),(0,3)):maior,((1,4),(1,3)):menor,((6,4),(6,3)):menor, ((3,4),(4,4)):menor,((4,4),(5,4)):menor,((3,6),(3,5)):maior, ((3,5),(3,4)):menor, ((0,6),(0,5)): maior, ((0,2),(1,2)): maior}

futoshiki7Extreme = futoshiki(7, preenchidos7ext, desigualdades7ext)
start = timeit.default_timer()
r = backtracking_search(futoshiki7Extreme)
stop = timeit.default_timer()
print(f"Tempo de execução: {stop - start} segundos")

Tempo de execução: 78.1610335999867


Futoshiki 9x9 dificuldade extrema de acordo com o website  

Isto foi desperdicio de tempo pq nao esta a encontrar solucao em tempo util :(

In [75]:
preenchidos9x9 = {(0,0) :  [6], (6,0) : [3], (2,1) : [2], (5,4) : [4], (2,6) : [3], (0,7) : [4], \
                  (2,8) : [6], (4,8) : [5], (8,8) : [8]}

# Quero chorar
desigualdades9x9 = {((0,0),(1,0)) : menor, ((1,1),(1,0)) : maior, ((3,1),(4,1)) : maior, ((4,1),(5,1)) : maior, \
                    ((7,1),(7,0)) : menor, ((7,1),(8,1)) : maior, ((0,2),(0,1)) : maior, ((2,2),(2,1)) : maior, \
                    ((4,2),(4,1)) : maior, ((5,2),(6,2)) : menor, ((6,2),(6,1)) : menor, ((8,2),(8,1)) : menor, \
                    ((3,3),(3,2)) : menor, ((3,3),(4,3)) : menor, ((7,3),(7,2)) : menor, ((8,3),(8,2)) : menor, \
                    ((0,4),(1,4)) : menor, ((2,4),(3,4)) : maior, ((4,4),(4,3)) : maior, ((6,4),(7,4)) : menor, \
                    ((7,4),(8,4)) : menor, ((0,5),(1,5)) : maior, ((1,5),(1,4)) : maior, ((7,4),(7,3)) : menor, \
                    ((6,5),(7,5)) : maior, ((7,5),(8,5)) : menor, ((3,6),(4,6)) : maior, ((5,6),(5,5)) : menor, \
                    ((6,6),(7,6)) : maior, ((7,6),(7,5)) : menor, ((7,6),(8,6)) : menor, ((0,7),(0,6)) : menor, \
                    ((3,7),(4,7)) : menor, ((4,7),(4,6)) : menor, ((7,7),(7,6)) : menor, ((1,8),(2,8)) : maior, \
                    ((2,8),(2,7)) : menor, ((4,8),(4,7)) : menor, ((7,8),(8,8)) : maior}

In [76]:
futoshiki9 = futoshiki(9, preenchidos9x9, desigualdades9x9)

In [77]:
visualizacao_futoshiki(9, futoshiki9, desigualdades9x9, {})

Estado Inicial:
6 < 0   0   0   0   0   3   0   0
    ∧                       ∨    
0   0   2   0 > 0 > 0   0   0 > 0
∧       ∧       ∧       ∨       ∨
0   0   0   0   0   0 < 0   0   0
            ∨               ∨   ∨
0   0   0   0 < 0   0   0   0   0
                ∧           ∨    
0 < 0   0 > 0   0   4   0 < 0 < 0
    ∧                            
0 > 0   0   0   0   0   0 > 0 < 0
                    ∨       ∨    
0   0   3   0 > 0   0   0 > 0 < 0
∨               ∨           ∨    
4   0   0   0 < 0   0   0   0   0
        ∨       ∨                
0   0 > 6   0   5   0   0   0 > 8
                                 

Solução:
6 < 0   0   0   0   0   3   0   0
    ∧                       ∨    
0   0   2   0 > 0 > 0   0   0 > 0
∧       ∧       ∧       ∨       ∨
0   0   0   0   0   0 < 0   0   0
            ∨               ∨   ∨
0   0   0   0 < 0   0   0   0   0
                ∧           ∨    
0 < 0   0 > 0   0   4   0 < 0 < 0
    ∧                            
0 > 0   0   0   0   0 

In [78]:
start  =timeit.default_timer()
r = backtracking_search(futoshiki9)
stop = timeit.default_timer()
print(f"Tempo de execução: {stop - start}")

KeyboardInterrupt: 

## Visualização do problema

In [110]:
def visualizacao_futoshiki(n, problema_csp, desigualdades, solucao):
    
    def mapeamento_2D_para_1D(x,y):
        return y*(n+(n-1)*3+1) + x

    def escrita_numeros(x,y):
        return mapeamento_2D_para_1D(x*4, y*2)
    
    def escrita_desigualdades_linha(x,y):
        return mapeamento_2D_para_1D(x*4+2, y*2)
    
    def escrita_desigualdades_coluna(x,y):
        return mapeamento_2D_para_1D(x*4, y*2-1)

    def escrever_desigualdade(operacao, direcao):
        if direcao == "x":
            if operacao.__name__ == "maior":
                return '>'
            else:
                return '<'
        else:
            if operacao.__name__ == "maior":
                return u"\u2227" # Carater ∧
            else:
                return u"\u2228" # Carater ∨

    board_inicial = ([" "] * (n+(n-1)*3) + ["\n"]) * (n*2)
    for var, dom in problema_csp.domains.items():
        if len(dom) == 1:
            board_inicial[escrita_numeros(var[0],var[1])] = str(dom[0])
        else:
            board_inicial[escrita_numeros(var[0],var[1])] = "0"
    for var, operacao in desigualdades.items():
        if (var[0][1] == var[1][1]):
            board_inicial[escrita_desigualdades_linha(var[0][0],var[0][1])] = escrever_desigualdade(operacao, "x")
        else:
            board_inicial[escrita_desigualdades_coluna(var[0][0],var[0][1])] = escrever_desigualdade(operacao, "y")
    
    board_final = board_inicial.copy()
    for var, valor in solucao.items():
        board_final[escrita_numeros(var[0],var[1])] = str(valor)
    

    print("Estado Inicial:")
    print(''.join(board_inicial))
    print("Solução:")
    print(''.join(board_final))

In [17]:
visualizacao_futoshiki(5, futoshiki5, desigualdades, backtracking_search(futoshiki5))

Estado Inicial:
0   0   0   0 > 0
                 
0 < 0   0   0   0
                 
0 < 0   0   0   0
            ∨    
2   0   0   0   3
        ∧       ∧
0   0 > 0   0   0
                 

Solução:
5   1   4   3 > 2
                 
3 < 4   5   2   1
                 
1 < 2   3   5   4
            ∨    
2   5   1   4   3
        ∧       ∧
4   3 > 2   1   5
                 



## Coisas Extras

In [ ]:
preenchidos6 =

futoshiki6 = futoshiki(6, preenchidos6, desigualdades6)